Importing Libraries

In [10]:
from pprint import pprint
import os
import pandas as pd
import numpy as np
import warnings
import jsonschema_specifications
import json
warnings.filterwarnings("ignore")


Reading the data

In [11]:
events = pd.read_json(r"E:\customer_churn_prediction\data\interim\events.json", lines = True)
full_events = pd.read_json(r"E:\customer_churn_prediction\data\interim\cleaned_test_data.json", lines = True)

In [12]:
print("events:", events.shape)
print("full_events:", full_events.shape)

events: (278154, 18)
full_events: (248377, 18)


In [13]:
events.columns

Index(['ts', 'userId', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'location', 'userAgent', 'lastName',
       'firstName', 'registration', 'gender', 'artist', 'song', 'length'],
      dtype='object')

In [14]:
full_events.columns

Index(['ts', 'userId', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'location', 'userAgent', 'lastName',
       'firstName', 'registration', 'gender', 'artist', 'song', 'length'],
      dtype='object')

## Create Churn Label

In [17]:
if 'churn' not in events.columns:
    events_sorted = events.sort_values(['userId', 'ts'])
    user_last_events = events_sorted.groupby('userId').tail(1).copy()
    user_last_events['churn'] = (user_last_events['page'] == 'Cancellation Confirmation').astype(int)
    user_churn_map = user_last_events[['userId', 'churn']].set_index('userId')['churn'].to_dict()
    events['churn'] = events['userId'].map(user_churn_map).fillna(0).astype(int)
else:
    # ensure mapping is consistent
    events_sorted = events.sort_values(['userId', 'ts'])
    user_last_events = events_sorted.groupby('userId').tail(1).copy()

print("unique users (events):", events['userId'].nunique())

unique users (events): 225


In [18]:
events.columns

Index(['ts', 'userId', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'location', 'userAgent', 'lastName',
       'firstName', 'registration', 'gender', 'artist', 'song', 'length',
       'churn'],
      dtype='object')

In [19]:
full_events.columns

Index(['ts', 'userId', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'location', 'userAgent', 'lastName',
       'firstName', 'registration', 'gender', 'artist', 'song', 'length'],
      dtype='object')

### Reassuring that there is no overlap between the training and testing data (Made before but I will check again)

In [20]:
train_user_ids = set(events['userId'].unique())
test_user_ids = set(full_events['userId'].unique())

overlap = train_user_ids.intersection(test_user_ids)
print("overlap count:", len(overlap))


if len(overlap) > 0:
    full_events_clean = full_events[~full_events['userId'].isin(overlap)].copy()
    print(f"Removed {len(overlap)} overlapping users from full_events -> full_events_clean shape: {full_events_clean.shape}")
else:
    full_events_clean = full_events.copy()
    print("No overlap found.")


overlap count: 0
No overlap found.


In [21]:
events.churn.value_counts()

churn
0    233290
1     44864
Name: count, dtype: int64

In [23]:
if 'churn' not in events.columns or events['churn'].isnull().any():
    ev_sorted = events.sort_values(['userId','ts'])
    last = ev_sorted.groupby('userId').tail(1).copy()
    user_churn = last.set_index('userId')['page'].eq('Cancellation Confirmation').astype(int)
else:
    user_churn = events.groupby('userId').tail(1).set_index('userId')['churn']

print("train user churn value counts:\n", user_churn.value_counts())

train user churn value counts:
 churn
0    173
1     52
Name: count, dtype: int64


In [24]:
# test user churn
fe_sorted = full_events_clean.sort_values(['userId','ts'])
fe_last = fe_sorted.groupby('userId').tail(1).copy()
test_user_churn = fe_last.set_index('userId')['page'].eq('Cancellation Confirmation').astype(int)
print("test user churn value counts:\n", test_user_churn.value_counts())

test user churn value counts:
 page
0    176
1     48
Name: count, dtype: int64


In [25]:
# 3) events per user distribution (basic stats + tails)
ev_per_user = events.groupby('userId').size()
print("events per user (train) — mean, median, min, max:", ev_per_user.mean(), ev_per_user.median(), ev_per_user.min(), ev_per_user.max())
print("how many users with only 1 event:", (ev_per_user==1).sum())
print("events-per-user quantiles:\n", ev_per_user.quantile([0.0,0.25,0.5,0.75,0.9,0.99]))

events per user (train) — mean, median, min, max: 1236.24 848.0 6 9632
how many users with only 1 event: 0
events-per-user quantiles:
 0.00       6.00
0.25     296.00
0.50     848.00
0.75    1863.00
0.90    2514.00
0.99    6604.48
dtype: float64


In [27]:
# 4) churn positives per prospective CV fold (approx)
n_users = len(train_user_ids)
churn_pos = user_churn.sum()
print("total train users:", n_users, "total churn positives:", churn_pos, "positive rate:", churn_pos/n_users)
for n_splits in (3,4,5,10):
    approx_pos_per_fold = churn_pos / n_splits
    print(f"approx pos per fold if n_splits={n_splits}: {approx_pos_per_fold:.2f}")

total train users: 225 total churn positives: 52 positive rate: 0.2311111111111111
approx pos per fold if n_splits=3: 17.33
approx pos per fold if n_splits=4: 13.00
approx pos per fold if n_splits=5: 10.40
approx pos per fold if n_splits=10: 5.20


In [28]:
# 5) sanity: users that have cancellation event but more events after it
canc_users = events[events['page']=='Cancellation Confirmation']['userId'].unique()
after_cancel = []
for u in canc_users:
    uev = events[events['userId']==u].sort_values('ts')
    canc_idx = uev[uev['page']=='Cancellation Confirmation'].index.min()
    if canc_idx < uev.index.max():  # there are events after the first cancellation
        after_cancel.append(u)
print("users with cancellation then later events:", len(after_cancel))


users with cancellation then later events: 0


## Building user-level features

In [9]:
def build_user_features(events_df):
    # events_df: event-level data (must include 'userId', 'ts', 'song','artist','length','sessionId','itemInSession','page','level')
    df_sorted = events_df.sort_values(['userId', 'ts']).copy()
    # get each user's last-event index
    last_idx = df_sorted.groupby('userId').tail(1).index
    # events without the last event for each user (these will be used to compute features)
    df_no_last = df_sorted.drop(index=last_idx)
    # For users who only had 1 event, dropping leaves 0 rows -> we'll fill missing features later
    agg = df_no_last.groupby('userId').agg(
        total_events=('ts', 'count'),
        n_sessions=('sessionId', pd.Series.nunique),
        unique_songs=('song', pd.Series.nunique),
        unique_artists=('artist', pd.Series.nunique),
        total_length=('length', 'sum'),
        avg_length=('length', 'mean'),
        avg_itemInSession=('itemInSession', 'mean'),
        num_cancellation_events=('page', lambda s: (s == 'Cancellation Confirmation').sum()),
        num_thumbup=('page', lambda s: (s == 'Thumbs Up').sum()),
        num_thumbdown=('page', lambda s: (s == 'Thumbs Down').sum()),
        num_home=('page', lambda s: (s == 'Home').sum()),
        num_nextsong=('page', lambda s: (s == 'NextSong').sum()),
        last_level=('level', lambda s: s.iloc[-1] if len(s)>0 else np.nan),
        last_location=('location', lambda s: s.iloc[-1] if len(s)>0 else np.nan),
        first_ts=('ts', 'min'),
        last_ts_excluded=('ts', 'max')
    ).reset_index()

    # compute activity span in seconds (last - first)
    agg['activity_span_seconds'] = (agg['last_ts_excluded'] - agg['first_ts']).fillna(0)
    # fill NA numeric features with 0
    numeric_cols = ['total_events','n_sessions','unique_songs','unique_artists','total_length','avg_length',
                    'avg_itemInSession','num_cancellation_events','num_thumbup','num_thumbdown','num_home','num_nextsong','activity_span_seconds']
    agg[numeric_cols] = agg[numeric_cols].fillna(0)
    # last_level / last_location: fill missing with 'unknown'
    agg['last_level'] = agg['last_level'].fillna('unknown')
    agg['last_location'] = agg['last_location'].fillna('unknown')
    return agg

In [10]:
# Build training user features
train_user_feats = build_user_features(events)
# get labels from true last event per user
train_labels = user_last_events[['userId','churn']].drop_duplicates(subset=['userId']).set_index('userId')
train_user_feats = train_user_feats.set_index('userId').join(train_labels, how='left')
train_user_feats['churn'] = train_user_feats['churn'].fillna(0).astype(int)
train_user_feats = train_user_feats.reset_index()


In [11]:
print("train_user_feats:", train_user_feats.shape)
train_user_feats.head()

train_user_feats: (225, 19)


,userId,total_events,n_sessions,unique_songs,unique_artists,total_length,avg_length,avg_itemInSession,num_cancellation_events,num_thumbup,num_thumbdown,num_home,num_nextsong,last_level,last_location,first_ts,last_ts_excluded,activity_span_seconds,churn
0,2,898,7,713,587,188687.38342,249.917064,137.935412,0,29,6,35,755,paid,"Raleigh, NC",1538974195000,1542839403000,3865208000,0
1,3,253,4,211,197,54424.74544,254.321240,46.592885,0,14,3,8,214,paid,"Bozeman, MT",1538532534000,1540874979000,2342445000,1
2,4,2441,22,1799,1342,506140.04138,247.138692,152.165916,0,95,26,80,2048,paid,"Baltimore-Columbia-Towson, MD",1538356650000,1543595542000,5238892000,0
3,5,217,6,159,154,39525.04698,245.497186,28.193548,0,11,0,14,161,free,"Phoenix-Mesa-Scottsdale, AZ",1538581469000,1541637200000,3055731000,0
4,6,3760,24,2678,1868,787236.52359,249.204344,172.839362,0,165,31,131,3159,paid,"Houston-The Woodlands-Sugar Land, TX",1538406703000,1543531621000,5124918000,0


### Build test user features

In [12]:
full_sorted = full_events_clean.sort_values(['userId', 'ts']).copy()
test_user_last = full_sorted.groupby('userId').tail(1).copy()
test_user_last['churn'] = (test_user_last['page'] == 'Cancellation Confirmation').astype(int)

test_user_feats = build_user_features(full_events_clean)
test_labels = test_user_last[['userId','churn']].drop_duplicates(subset=['userId']).set_index('userId')
test_user_feats = test_user_feats.set_index('userId').join(test_labels, how='left')
test_user_feats['churn'] = test_user_feats['churn'].fillna(0).astype(int)
test_user_feats = test_user_feats.reset_index()

In [13]:
print("test_user_feats:", test_user_feats.shape)
test_user_feats.head()

test_user_feats: (224, 19)


,userId,total_events,n_sessions,unique_songs,unique_artists,total_length,avg_length,avg_itemInSession,num_cancellation_events,num_thumbup,num_thumbdown,num_home,num_nextsong,last_level,last_location,first_ts,last_ts_excluded,activity_span_seconds,churn
0,48,380,4,287,264,78848.35933,261.087283,69.744737,0,19,6,15,302,paid,"Jackson, MS",1538681898000,1541092581000,2410683000,0
1,130,363,7,264,249,71482.73064,262.804157,37.752066,0,14,2,16,272,free,"Bridgeport-Stamford-Norwalk, CT",1538482298000,1538890857000,408559000,1
2,157,2965,20,2174,1585,617699.17649,247.079671,188.997976,0,115,27,86,2500,paid,"Tullahoma-Manchester, TN",1538744376000,1543292957000,4548581000,0
3,158,130,4,89,87,23648.74337,265.716218,39.984615,0,5,2,10,89,free,"New York-Newark-Jersey City, NY-NJ-PA",1539238077000,1542605535000,3367458000,0
4,159,417,12,306,287,80522.48812,254.014158,34.462830,0,15,4,21,317,free,"New Orleans-Metairie, LA",1538543900000,1543577819000,5033919000,0


In [ ]:
out_dir = r"E:\customer_churn_prediction\data\processed"
expected = {
    "train_user_feats": globals().get("train_user_feats", None),
    "test_user_feats": globals().get("test_user_feats", None)
}

for name, df in expected.items():
    if df is None:
        raise RuntimeError(f"DataFrame '{name}' not found in the notebook. Make sure you ran the feature-building cells first.")
    # normalize index (save userId as column if it's the index)
    if df.index.name == 'userId' or (df.index.name is None and 'userId' not in df.columns and df.index.dtype == object):
        df_to_save = df.reset_index()
    else:
        df_to_save = df.copy()

    # file paths
    csv_path = os.path.join(out_dir, f"{name}.csv")
    jsonl_path = os.path.join(out_dir, f"{name}.jsonl")
    parquet_path = os.path.join(out_dir, f"{name}.parquet")
    meta_path = os.path.join(out_dir, f"{name}_metadata.json")

    # save
    df_to_save.to_csv(csv_path, index=False)
    df_to_save.to_json(jsonl_path, orient="records", lines=True)
    try:
        df_to_save.to_parquet(parquet_path, index=False)
    except Exception as e:
        # parquet may require pyarrow or fastparquet; warn but continue
        print(f"Warning: could not write parquet for {name} ({e}). Install pyarrow or fastparquet to enable parquet saving.")

    # metadata
    meta = {
        "rows": int(df_to_save.shape[0]),
        "cols": int(df_to_save.shape[1]),
        "columns": list(map(str, df_to_save.columns)),
        "csv": csv_path,
        "jsonl": jsonl_path,
        "parquet": parquet_path if os.path.exists(parquet_path) else None
    }
    with open(meta_path, "w", encoding="utf-8") as fh:
        json.dump(meta, fh, indent=2)

    print(f"Saved {name}: rows={meta['rows']}, cols={meta['cols']}")
    print(f"  csv -> {csv_path}")
    print(f"  jsonl -> {jsonl_path}")
    if meta["parquet"]:
        print(f"  parquet -> {parquet_path}")
    print(f"  metadata -> {meta_path}")
    print("----")


Saved train_user_feats: rows=225, cols=19
  csv -> E:\customer_churn_prediction\data\processed\train_user_feats.csv
  jsonl -> E:\customer_churn_prediction\data\processed\train_user_feats.jsonl
  parquet -> E:\customer_churn_prediction\data\processed\train_user_feats.parquet
  metadata -> E:\customer_churn_prediction\data\processed\train_user_feats_metadata.json
----
Saved test_user_feats: rows=224, cols=19
  csv -> E:\customer_churn_prediction\data\processed\test_user_feats.csv
  jsonl -> E:\customer_churn_prediction\data\processed\test_user_feats.jsonl
  parquet -> E:\customer_churn_prediction\data\processed\test_user_feats.parquet
  metadata -> E:\customer_churn_prediction\data\processed\test_user_feats_metadata.json
----
